# TP2 : MCMC (Partie 2)

## G3 SDI - Estimation Avancée

Dans ce TP, on s'intéresse aux méthodes d'échantillonnage dites "MCMC" (Monte Carlo par Chaînes de Markov). Le premier exercice consiste à implémenter un Metropolis-Hastings et de regarder l'influence de quelques paramètres. Dans un deuxième exercice, on cherchera à implémenter un échantillonneur de Gibbs dans un modèle de régression linéaire bayésienne.

### Instructions

1. Renommez votre notebook sous la forme `tp2b_Nom1_Nom2.ipynb`. 

2. Votre code, ainsi que toute sortie du code, doivent être commentés !

3. Déposez votre notebook sur Moodle dans la section prévue à cet effet avant la date limite.

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Compte-rendu écrit par [nom1], [nom2], date.
</div>

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats as ss

### Exercice - Bayesian LASSO

Soit le modèle hiérarchique suivant, correspondant à un modèle de régression linéaire bayésienne :
\begin{align}
\sigma^2 & \sim \text{InverseGamma}(a_0, b_0) & \\
\tau_j & \sim \text{InverseGamma} \left(1, \frac{\lambda^2}{2} \right) \qquad & \forall~j \in {1, ..., p} \\
\beta_j | \tau_j, \sigma^2 & \sim \mathcal{N} \left( 0, \frac{\sigma^2}{\tau_j} \right) \qquad & \forall~j \in {1, ..., p} \\
y_i | \boldsymbol{x}_i, \boldsymbol{\beta}, \sigma^2 & \sim \mathcal{N}(\boldsymbol{\beta}^{\top} \mathbf{x}_i, \sigma^2) \qquad & \forall~i \in {1, ..., n} \\
\end{align}

On a :
* $y_i \in \mathbb{R}$ la variable que l'on cherche à prédire ;
* $\mathbf{x}_i \in \mathbb{R}^p$ les features ;
* $\boldsymbol{\beta} \in \mathbb{R}^p$ le vecteur de régression ;
* $\sigma^2$ la variance du bruit.
* $\boldsymbol{\tau} \in \mathbb{R}^p$ une variable latente du modèle.

$\lambda$, $a_0$, et $b_0$ sont quant à eux supposés connus et fixés.
  
Ce modèle s'appelle le *Bayesian LASSO* ([Park et al. (2008)](https://people.eecs.berkeley.edu/~jordan/courses/260-spring09/other-readings/park-casella.pdf)) (le .pdf de l'article est dans l'archive). Pour rappel, le LASSO correspond à la régression linéaire pénalisé en norme $\ell_1$.

À partir des données $\mathcal{D} = \{ (\mathbf{x}_1, y_1), ... (\mathbf{x}_n, y_n) \}$, l'objectif est de caractériser la loi a posteriori $p(\boldsymbol{\beta}, \boldsymbol{\tau}, \sigma^2 | \mathcal{D})$. Elle n'est pas tractable analytiquement. Nous allons donc échantillonner de cette loi a posteriori à l'aide d'un algorithme MCMC, plus précisémment, un [échantillonneur de Gibbs](https://fr.wikipedia.org/wiki/%C3%89chantillonnage_de_Gibbs).

On peut trouver les lois conditionnelles dans ce modèle :
\begin{align}
\boldsymbol{\beta} | \boldsymbol{\tau}, \sigma^2, \mathcal{D} \sim \mathcal{N}(\mathbf{A} \mathbf{X}^{\top} \mathbf{y}, \sigma^2 \mathbf{A}),
\end{align}
avec $\mathbf{A} = (\mathbf{X}^{\top} \mathbf{X} + \mathbf{D}_{\tau})^{-1}$, où $\mathbf{D}_{\tau} = \text{diag}(\tau_1, ..., \tau_p)$.
\begin{align}
\tau_j | \boldsymbol{\beta}, \sigma^2, \mathcal{D} \sim \text{InverseGaussian}\left( \frac{\lambda \sigma}{\beta_j}, \lambda^2 \right),
\end{align}
(Loi "Inverse-Gaussian" -> voir [ici](https://en.wikipedia.org/wiki/Inverse_Gaussian_distribution))
\begin{align}
\sigma^2 | \boldsymbol{\beta}, \boldsymbol{\tau}, \mathcal{D} \sim \text{InverseGamma}\left( a_0 + \frac{n+p}{2},~b_0 + \frac{1}{2} || \mathbf{y - X} \boldsymbol{\beta} ||^2_2 + \frac{1}{2} \boldsymbol{\beta}^{\top} \mathbf{D}_{\tau} \boldsymbol{\beta}  \right).
\end{align}

In [ ]:
# Dataset

from sklearn.datasets import load_diabetes

diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target

X = X*np.sqrt(442) # Standardize, X is already centered
y = (y-np.mean(y))/np.std(y) # Zero-mean output, as no intercept in the model for simplicity

**Q1.** Écrire une fonction implémentant l'échantillonnage de Gibbs dans ce modèle, qui prend en arguments :
* Les données sous forme matricielle $\mathbf{X}$ et $\mathbf{y}$ ;
* Le nombre d'échantillons après burn-in $N_g$ ;
* La taille du *burn-in* $N_b$ ;
* Les hyperparamètres du modèle : $a_0,~b_0,~\lambda$.

Elle retourne $N_g$ échantillons du posterior $p(\boldsymbol{\beta}, \boldsymbol{\tau}, \sigma^2 | \mathcal{D})$. On pourra initialiser la chaîne en tirant de la loi a priori.

NB :
* Pour échantillonner d'une loi inverse gamma (a,b), on pourra utiliser `invgamma.rvs(a, loc=0, scale=b)`
* Pour échantillonner d'une loi inverse gaussian (m,l), on pourra utiliser `invgauss.rvs(m/l, loc=0, scale=l)`

In [ ]:
def bayesian_lasso_gibbs(X, y, Ng, Nb, a0, b0, l):
    #######
    ## YOUR CODE HERE
    #######

**Q2.** Faire tourner la chaîne MCMC avec les paramètres suivants : $N_g = 2000$, $N_b = 1000$, $a_0 = 1.5$, $b_0 = 1$, $\lambda = 10$.

* Afficher les *traceplots* pour $\sigma^2$ et quelques-uns des $\beta_p$. Commenter
* Pour tous les $\boldsymbol{\beta}$ et $\sigma^2$, afficher un histogramme ou un KDE des $N_g$ échantillons obtenus. Afficher sur le même plot la valeur de l'estimation MMSE. Enfin, donner l'intervalle de crédibilité à 95%.

In [ ]:
Ng = 2000
Nb = 1000
a0 = 1.5
b0 = 1
l = 10

#######
## YOUR CODE HERE
#######

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>

**Q3.** Générer 1000 échantillons de la loi prédictive a posteriori pour le nouveau point $x_{\text{new}}$ (défini dans le code) et tracer un histogramme ou un KDE. Commenter.

In [ ]:
x_new = np.array([ 1.41145807,  1.06548848,  0.30006161,  0.45984057, -0.52475728,
       -1.70643289,  1.02259953,  1.49710409, -1.25030999,  0.84817082])

#######
## YOUR CODE HERE
#######

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>

**Q4.** Étudier l'influence du paramètre $\lambda$ sur l'inférence des paramètres (prendre e.g. $\lambda = 1$ et $\lambda=100$). Commenter.

In [ ]:
#######
## YOUR CODE HERE
#######

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>